In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Import the data into a Pandas DataFrame
ward_df = pd.read_csv("resources/Census_stats_consolidated_ward.csv")
ward_df.head()

,Ward,2016_Owned,2016_Rental,2016_couples_with_children,2016_lone_parent_with_children,2016_English,2016_French,2016_Other,2016_no certificate,2016_high school,...,2011_English,2011_French,2011_Other,2011_no certificate,2011_high school,2011_post_secondary,2010_average_income,2010_median_income,2010_HH_in low_income(after tax),yoy_median_income_change
0,Ward 1,9920,8385,3680,4285,25355,340,30955,13270,14750,...,25000,305,32875,12300,15170,20515,66001,55027,13355,9.7
1,Ward 2,11700,7765,4300,4200,26530,425,26925,10400,14280,...,26140,440,26990,11180,12715,20875,72100,57718,10465,11.8
2,Ward 3,14280,5780,4840,3085,27405,550,22915,6515,11575,...,27485,490,22125,6590,11020,25860,95259,72175,6805,13.4
3,Ward 4,12630,9155,5420,3145,29570,440,23540,7500,12130,...,28490,565,23950,7815,11690,26280,106513,69501,6890,13.1
4,Ward 5,19050,11415,7745,3240,38885,1080,27675,6825,12845,...,35575,870,25555,6650,12065,35360,110919,75035,7470,6.7


In [3]:
crime2014_df = pd.read_csv("resources/MCI_2014.csv")
crime2014_df.head()

,reportedyear,MCI,Neighbourhood,Lat,Long
0,2014,Robbery,North Riverdale (68),43.668449,-79.343094
1,2014,Break and Enter,Black Creek (24),43.759285,-79.507927
2,2014,Assault,Brookhaven-Amesbury (30),43.697552,-79.501663
3,2014,Assault,Rexdale-Kipling (4),43.721703,-79.571510
4,2014,Assault,Lambton Baby Point (114),43.663891,-79.503487


In [4]:
crime2018_df = pd.read_csv("resources/MCI_2018.csv")
crime2018_df.head()

,reportedyear,MCI,Division,Neighbourhood,Lat,Long
0,2018,Assault,D42,Agincourt North (129),43.796127,-79.261513
1,2018,Assault,D42,Agincourt North (129),43.796127,-79.261513
2,2018,Assault,D42,Agincourt North (129),43.797062,-79.283440
3,2018,Assault,D42,Agincourt North (129),43.797409,-79.281662
4,2018,Assault,D42,Agincourt North (129),43.797409,-79.281662


In [5]:
engine = create_engine(f'sqlite:///C:\\Users\\haiye\\Desktop\\Data_Bootcamp_Project_03\\SQL_database\\to_ward_project.sqlite')

In [6]:
ward_df.to_sql(name='ward_data', con=engine, if_exists='append', index=True)

In [7]:
crime2014_df.to_sql(name='crime2014_data', con=engine, if_exists='append', index=True)

In [8]:
crime2018_df.to_sql(name='crime2018_data', con=engine, if_exists='append', index=True)